In [4]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import re

In [14]:
years = np.arange(2018, 2024)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
path_base = Path('data')
directories = [path_base / str(year) / month for year in years for month in months]

dfs = []
for dir in directories:
    if dir.exists():
        for file in dir.iterdir():
            if file.is_file() and file.name.startswith('y'):
                df = wrangle(file)
                dfs.append(df)

dfs[0]


,Parameter,Site Name,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,Avg,Max,Hr. of Max,Date
3,01 OZONE PPM,ALPINE,0.049,NaN,NaN,0.047,0.048,0.045,0.043,0.043,...,0.044,0.042,0.042,0.044,0.042,0.046,0.045,0.056,14,2018
4,01 OZONE PPM,CHULA VISTA,0.002,NaN,NaN,0.002,0.004,0.004,0.002,0.003,...,0.021,0.011,0.003,0.004,0.012,0.016,0.019,0.048,13,2018
5,01 OZONE PPM,DEL_MAR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2018
6,01 OZONE PPM,DOWNTOWN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2018
7,01 OZONE PPM,EL CAJON LES,0.004,NaN,NaN,NaN,0.003,0.003,0.003,0.004,...,0.019,0.009,0.006,0.004,0.003,0.003,0.020,0.052,14,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,17 RELHUM %,KEARNY MESA,99.000,95.0,95.0,93.000,97.000,95.000,89.000,85.000,...,95.000,96.000,99.000,96.000,94.000,90.000,81.000,99.000,00,2018
108,18 PRES mmHg,KEARNY MESA,752.000,752.0,752.0,752.000,752.000,752.000,752.000,752.000,...,752.000,752.000,752.000,753.000,753.000,753.000,752.000,753.000,08,2018
109,19 SOLAR W/M2,KEARNY MESA,1.000,0.0,0.0,0.000,1.000,1.000,3.000,93.000,...,1.000,1.000,1.000,1.000,1.000,1.000,145.000,591.000,10,2018
110,28 SO2 Tr PPB,EL CAJON LES,0.400,NaN,NaN,NaN,0.200,0.200,0.200,0.200,...,0.100,0.100,0.100,0.200,0.100,0.100,0.100,0.400,00,2018


In [8]:
each_row_year['Parameter'] = each_row_year['Parameter'].str.replace(r'^\d+\s?', '', regex=True)
each_row_year =(each_row_year.groupby('Parameter')
    .filter(lambda df: df.shape[0] > 3))
each_row_year[each_row_year['Parameter'] == 'CO PPM']

NameError: name 'each_row_year' is not defined

In [14]:
each_row_year['Parameter']

array(['BCarbon ug/m3', 'OZONE PPM', 'NOX PPM', 'NO2 PPB', 'NO2 PPM',
       'NO PPB', 'NO PPM', 'INTMP °C', 'PM2.5 µg/M3', 'VWDR °',
       'SIGTHETA °', 'VWSP MPH', 'SWSP MPH', 'AMBTEMP °F', 'RELHUM %',
       'PM10 STD', 'CO PPM', 'PRES mmHg', 'SOLAR W/M2', 'UVRAD W/M2',
       'NOY', 'NOY-NO', 'NO Trace', 'SO2 Tr PPB', 'CO Tr PPB', 'RAINFALL'],
      dtype=object)

In [37]:
biggie = pd.concat(dfs)

biggie['Avg'] = biggie['Avg'].replace({'':np.nan, 'M':np.nan, '<':np.nan, 'P':np.nan})
biggie['Avg'] = biggie['Avg'].astype(float)
biggie['Site Name'] = biggie['Site Name'].str.title()


grouped  = biggie.groupby(['Site Name', 'Parameter', 'Date'])['Avg'].mean()
each_row_year = grouped.reset_index()
each_row_year['Parameter'].unique()
each_row_year['Parameter'] = each_row_year['Parameter'].str.replace(r'^\d+\s?', '', regex=True)
each_row_year['Site Name'] = each_row_year['Site Name'].str.replace('Downtown Ses', 'Downtown')

for year in each_row_year['Date'].unique():
    this_year = each_row_year[each_row_year['Date'] == year]
    for place in this_year['Site Name'].unique():
        year_place = this_year[this_year['Site Name'] == place]
        year_place = year_place.to_json(orient = 'records')
        with open(year+'/'+place+'.json', 'w') as outfile:
            json.dump(year_place, outfile)



# each_row_json = each_row.to_json(orient='records')

# # Write JSON to a file
# with open("everything.json", "w") as outfile:
#     json.dump(each_row_json, outfile)
each_row_year['Parameter'].value_counts()

Parameter
INTMP °C         68
AMBTEMP °F       64
VWSP MPH         57
VWDR °           57
SWSP MPH         57
SIGTHETA °       57
OZONE PPM        56
PM2.5 µg/M3      51
NO PPM           46
NO2 PPM          46
NOX PPM          46
BCarbon ug/m3    32
NO2 PPB          27
PM10 STD         23
RELHUM %         21
NO PPB           15
CO PPM           14
PRES mmHg        13
SOLAR W/M2       13
CO Tr PPB         7
SO2 Tr PPB        7
RAINFALL          6
NOY               6
NOY-NO            6
NO Trace          6
UVRAD W/M2        5
BCarbon ng/m3     2
H2S PPB           2
Name: count, dtype: int64

In [36]:
each_row_year[each_row_year['Site Name'] == 'Alpine']

,Site Name,Parameter,Date,Avg
5,Alpine,OZONE PPM,2018,0.042657
6,Alpine,OZONE PPM,2019,0.038136
7,Alpine,OZONE PPM,2020,0.042478
8,Alpine,OZONE PPM,2021,0.044350
9,Alpine,OZONE PPM,2022,0.044495
...,...,...,...,...
90,Alpine,BCarbon ug/m3,2021,0.282861
91,Alpine,BCarbon ug/m3,2022,0.245803
92,Alpine,PM10 STD,2022,18.196491
93,Alpine,PM10 STD,2023,16.778630


In [5]:
import pandas as pd

# Read the file with custom parsing
file_path = 'data/2023/Apr/yesterday_20230402.CSV'
def wrangle(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            lines = file.readlines()
    
    # Extract date and irrelevant lines
    date_info = lines.pop(0).strip()
    year = re.search(r'\b\d{4}\b', date_info).group()

    lines.pop(0)
    
    # Initialize data structure
    data = []

    # Parse the remaining lines
    for line in lines:
        split_line = line.strip().split(',')
        if len(split_line) >= 29:  # Ensure there are at least 29 fields
            data.append(split_line[:29])  # Adjust if different number of fields per row

    # Create DataFrame from the parsed data
    columns = [
        'Parameter', 'Site Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
        '17', '18', '19', '20', '21', '22', '23', 'Avg', 'Max', 'Hr. of Max'
    ]
    hours = [str(x) for x in np.arange(0,24)]
    df = pd.DataFrame(data, columns=columns)
    df['Date'] =  year
    if year=='2018' or year == '2019':
        df = df.iloc[3:]
    else:
        df = df.iloc[1:]
    df['Avg'] = df['Avg'].replace({'':np.nan,'<':np.nan, 'M':np.nan, 'P':np.nan, '-':np.nan}).astype(float)
    df['Max'] = df['Max'].replace({'':np.nan,'<':np.nan, 'M':np.nan, 'P':np.nan, 'C':np.nan}).astype(float)

    for hour in hours:
        df[hour] = df[hour].replace({'':np.nan,'<':np.nan, 'M':np.nan, 'P':np.nan, 'C':np.nan, '-':np.nan}).astype(float)


    # Forward fill the 'Parameter' column
    df['Parameter'] = df['Parameter'].replace('',np.nan).ffill()
    return df

test= wrangle(file_path)

def highest_8_hour_avg(row):
    max_avg = 0
    for hour in  np.arange(0,17):
        # Extract the hourly values
        values = row.loc[str(hour):str(hour + 7)]
        values = values.dropna()
        # Check if we have at least 8 consecutive hours
        if len(values) >= 6:
            # Calculate the 8-hour average
            avg = np.mean(values)
            max_avg = max(max_avg, avg)
    
    return max_avg

# Sample DataFrame
test

,Parameter,Site Name,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,Avg,Max,Hr. of Max,Date
1,01 OZONE PPM,ALPINE,0.049,0.048,NaN,NaN,0.047,0.046,0.046,0.045,...,0.051,0.049,0.039,0.045,0.051,0.054,0.051,0.065,13,2023
2,01 OZONE PPM,CHULA VISTA,0.019,0.018,NaN,NaN,0.010,0.007,0.009,0.017,...,0.045,0.034,0.026,0.019,0.008,0.004,0.030,0.056,13,2023
3,01 OZONE PPM,DOWNTOWN SES,0.019,0.015,NaN,NaN,0.006,0.002,0.009,0.024,...,0.039,0.035,0.028,0.028,0.022,0.021,0.031,0.054,12,2023
4,01 OZONE PPM,EL CAJON LES,0.012,0.011,NaN,NaN,NaN,0.009,0.011,0.018,...,0.046,0.034,0.031,0.026,0.019,0.011,0.034,0.062,13,2023
5,01 OZONE PPM,KEARNY MESA,0.022,0.021,NaN,NaN,0.010,0.010,0.012,0.024,...,0.041,0.038,0.023,0.019,0.013,0.015,0.035,0.062,12,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,PM10 STD,DOWNTOWN SES,21.400,18.800,19.2,15.2,15.400,18.100,15.200,13.700,...,17.600,20.400,24.200,25.200,27.100,26.100,19.600,27.100,22,2023
90,PM10 STD,EL CAJON LES,10.600,17.400,7.9,8.6,10.600,8.000,9.100,10.600,...,11.700,18.100,22.600,15.600,18.400,19.900,12.600,22.600,20,2023
91,PM10 STD,OTAY MESA DVN,33.800,62.800,52.2,31.7,23.900,40.800,48.500,62.800,...,16.700,17.200,23.200,51.500,44.000,32.100,35.400,84.500,09,2023
92,PM10 STD,PENDLETON,15.600,14.800,14.0,13.6,13.000,11.300,11.800,14.500,...,19.400,23.600,19.500,25.400,15.800,15.700,16.700,25.400,21,2023


In [142]:
ozones = biggie[biggie['Parameter'] == '01 OZONE PPM']
ozones.loc[:, '8 hour max']=ozones.apply(highest_8_hour_avg, axis = 1)
ozones

C:\Users\banga\AppData\Local\Temp\ipykernel_24428\2761789762.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ozones.loc[:, '8 hour max']=ozones.apply(highest_8_hour_avg, axis = 1)


,Parameter,Site Name,0,1,2,3,4,5,6,7,...,19,20,21,22,23,Avg,Max,Hr. of Max,Date,8 hour max
3,01 OZONE PPM,Alpine,0.049,NaN,NaN,0.047,0.048,0.045,0.043,0.043,...,0.042,0.042,0.044,0.042,0.046,0.045,0.056,14,2018,0.049625
4,01 OZONE PPM,Chula Vista,0.002,NaN,NaN,0.002,0.004,0.004,0.002,0.003,...,0.011,0.003,0.004,0.012,0.016,0.019,0.048,13,2018,0.039500
5,01 OZONE PPM,Del_Mar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2018,0.000000
6,01 OZONE PPM,Downtown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,2018,0.000000
7,01 OZONE PPM,El Cajon Les,0.004,NaN,NaN,NaN,0.003,0.003,0.003,0.004,...,0.009,0.006,0.004,0.003,0.003,0.020,0.052,14,2018,0.042000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,01 OZONE PPM,Downtown Ses,0.005,NaN,NaN,0.014,0.016,0.020,0.015,0.010,...,0.020,0.011,0.012,0.011,0.006,0.021,0.037,15,2024,0.033500
4,01 OZONE PPM,El Cajon Les,0.011,NaN,NaN,NaN,0.004,0.010,0.005,0.007,...,0.007,0.004,0.006,0.006,0.007,0.018,0.040,13,2024,0.034500
5,01 OZONE PPM,Kearny Mesa,0.015,NaN,NaN,0.016,0.017,0.019,0.023,0.024,...,0.022,0.015,0.014,0.017,0.012,0.025,0.040,12,2024,0.035875
6,01 OZONE PPM,Otay Mesa Dvn,0.027,NaN,NaN,0.022,0.027,0.028,0.018,0.012,...,0.029,0.028,0.026,0.019,0.026,0.028,0.043,13,2024,0.037625


In [143]:
ozones[ozones['8 hour max'] > 0.070].groupby(['Date', 'Site Name']).size()

Date  Site Name    
2018  Alpine           24
      El Cajon Les      2
      Kearny Mesa       5
      Otay Mesa Dvn     1
2020  Alpine           29
      Chula Vista       4
      Downtown Ses      4
      El Cajon Les     15
      Kearny Mesa      12
      Otay Mesa Dvn    11
      Pendleton         3
2021  Alpine           15
      El Cajon Les      3
      Kearny Mesa       2
2022  Alpine           27
      El Cajon Les      2
      Kearny Mesa       2
      Otay Mesa Dvn     2
2023  Alpine           32
      Chula Vista       1
      Downtown Ses      1
      El Cajon Les      8
      Kearny Mesa       4
      Otay Mesa Dvn     2
      Pendleton         2
dtype: int64

In [146]:
ozones.groupby(['Date', 'Site Name'])['8 hour max'].mean()

Date  Site Name    
2018  Alpine           0.052992
      Chula Vista      0.039806
      Del_Mar          0.000000
      Downtown         0.000000
      El Cajon Les     0.046284
      Escondido        0.000000
      Kearny Mesa      0.043791
      Otay Mesa Dvn    0.044537
      Pendleton        0.038415
2019  Alpine           0.042911
      Chula Vista      0.038756
      Del_Mar          0.000000
      Downtown         0.000000
      El Cajon Les     0.039789
      Escondido        0.000000
      Kearny Mesa      0.038539
      Otay Mesa Dvn    0.036999
      Pendleton        0.038569
2020  Alpine           0.051763
      Chula Vista      0.041131
      Downtown Ses     0.041259
      El Cajon Les     0.046051
      Escondido        0.000000
      Kearny Mesa      0.045001
      Otay Mesa Dvn    0.044208
      Pendleton        0.041019
2021  Alpine           0.053989
      Chula Vista      0.042964
      Downtown Ses     0.041833
      El Cajon Les     0.046561
      Escondido     